In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import joblib
#from scipy import spatial
import numpy as np
#from transformers import pipeline, AutoTokenizer, TFPreTrainedModel  
#from tqdm import tqdm
from gensim.models import Word2Vec


df = joblib.load("../fydjob/output/indeed_proc/ip_2021-03-05.joblib")

In [9]:
df.head(5)

,job_title,job_text,company,location,job_info,query_text,source,job_link,tag_language,reviews,job_info_tokenized,job_text_tokenized,job_text_tokenized_titlecase,job_title_tokenized
0,(Junior) Data Engineer (f/m/x),Customlytics ist die führende App Marketing Be...,Customlytics GmbH,Berlin,(Junior) Data Engineer (f/m/x)\nCustomlytics G...,data science,scrape_json,NaN,en,NaN,"[junior, data, engineer, fmx, customlytics, gm...","[customlytics, ist, die, führende, app, market...","[Customlytics, ist, die, führende, App, Market...","[junior, data, engineer, fmx]"
1,,Responsibilities\n\nAs working student (m/f/x)...,Aroundhome,Berlin,Aroundhome6 Bewertungen - Berlin,data science,scrape_json,NaN,en,NaN,"[aroundhome, bewertungen, berlin]","[responsibilities, as, working, student, mfx, ...","[Responsibilities, As, working, student, mfx, ...",[]
2,,Aufgaben\nAls Werkstudent (m/w/d) IT arbeitest...,Aroundhome,Berlin,"Aroundhome6 Bewertungen - Berlin\nTeilzeit, Pr...",data science,scrape_json,NaN,de,NaN,"[aroundhome, bewertungen, berlin, teilzeit, pr...","[aufgaben, als, werkstudent, mwd, it, arbeites...","[Aufgaben, Als, Werkstudent, mwd, IT, arbeites...",[]
3,,Startdatum\n\nAb sofort\n\nDeine Aufgaben\n\nD...,Alexander Thamm GmbH,Berlin,Alexander Thamm GmbH - Berlin\nFestanstellung,data science,scrape_json,NaN,de,NaN,"[alexander, thamm, gmbh, berlin, festanstellung]","[startdatum, ab, sofort, deine, aufgaben, dein...","[Startdatum, Ab, sofort, Deine, Aufgaben, Dein...",[]
4,Full Stack Developer (m/f/d),We’re Phiture: a leading mobile growth consult...,Phiture,BerlinKreuzberg,Full Stack Developer (m/f/d)\nPhiture - Berlin...,data science,scrape_json,NaN,en,NaN,"[full, stack, developer, mfd, phiture, berlink...","[were, phiture, a, leading, mobile, growth, co...","[Were, Phiture, a, leading, mobile, growth, co...","[full, stack, developer, mfd]"


## Baseline Words2vec model

In [9]:
### instanciate model


w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=20,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     )

### building vocab with tokenized words
w2v_model.build_vocab(df["job_text_sent"], progress_per=10000) 


###training the model on the dataset
w2v_model.train(df["job_text_sent"], total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)


### most similar words example
w2v_model.wv.most_similar(["python"])


TypeError: unhashable type: 'list'

In [169]:
w2v_model.most_similar("python",topn = 5)

<ipython-input-169-cd99036e07f5>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  w2v_model.most_similar("python",topn = 5)


[('r', 0.956484317779541),
 ('java', 0.946160078048706),
 ('scala', 0.945618212223053),
 ('perl', 0.9335843324661255),
 ('matlab', 0.9326244592666626)]

## Training a Words2vec model with bi-gram parser

### Preprocessing

In [4]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from nltk.tokenize import sent_tokenize
'''
Preprocessing for the Job descriptions in paresed in senteneces.
Modified form the other preprocessing pipeline
'''
def to_lower(text):
    return text.lower()


def remove_number(text):
    text = ''.join(word for word in text if not word.isdigit())
    
    return text


def remove_punctuation_mod(text):
    
    punct = string.punctuation.replace(".","")
    for punctuation in punct:
        text = text.replace(punctuation, '')
    
    return text


def remove_stopwords(text):

    stop_words = set(stopwords.words('english'))
    new_text = []
    for sent in text:
        sent = word_tokenize(sent) 
        sent = [w for w in sent if w not in stop_words and w not in string.punctuation and w]  
        new_text.append(sent)
    return new_text


def lemmatize_words(text):
    
    lemmatizer = WordNetLemmatizer()
    for sentence in text:
        for word in sentence:
            word = lemmatizer.lemmatize(word)
        
    return text



In [5]:
### filter out german offers
df = df[df["tag_language"] == "en"]

In [6]:
### Apply preprocessing to df
df["job_text_sent"]= df["job_text"].apply(to_lower).apply(remove_number)\
                                    .apply(lambda x : x.replace('\n',' '))\
                                    .apply(remove_punctuation_mod)\
                                    .apply(lambda x: sent_tokenize(x))\
                                    .apply(remove_stopwords)\
                                    .apply(lemmatize_words)




In [8]:
df["job_text_sent"][10]

[['mbition',
  'mercedes–benz',
  'bringing',
  'automotive',
  'industry',
  'future',
  'working',
  'softwaredefined',
  'architecture',
  'allows',
  'us',
  'create',
  'stateoftheart',
  'customer',
  'functions',
  'span',
  'one',
  'domain',
  'controllers',
  'significant',
  'processing',
  'power'],
 ['one',
  'highperformance',
  'domain',
  'controllers',
  'used',
  'automated',
  'driving',
  'functions',
  'e.g'],
 ['automated', 'driving', 'highways'],
 ['berlin',
  'sindelfingen',
  'subsidiary',
  'mercedesbenz',
  'ag',
  'perfecting',
  'art',
  'moving',
  'people',
  'products',
  'services',
  'b',
  'years'],
 ['international',
  'team',
  'employees',
  'want',
  'integrate',
  'people',
  'appreciate',
  'quality',
  'creativity',
  'teamwork',
  'much'],
 ['together',
  'create',
  'passionate',
  'technology',
  'exciting',
  'topics',
  'like',
  'automated',
  'autonomous',
  'driving',
  'infotainment',
  'systems',
  'mobile',
  'apps'],
 ['need',
  'he

### Parse the sentences

In [10]:
### Turn df into list of sentences
sentences = df["job_text_sent"].tolist()

### reduce the nesting of the list to fit the format of the Phrases module
sentence = []
for second in sentences:
    for first in second:
        sentence.append(first)
        

In [11]:
### Train the phraser to detect bi-grams
from gensim.models.phrases import Phrases

phrases = Phrases(sentence, min_count=30, progress_per=10000)
### transform the list of sentences to detect bigrams
sent = []
for phrase in phrases[sentence]:
    sent.append(phrase)

### Word2vec model v2
    

In [12]:
from gensim.models import Word2Vec

w2v_model2 = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     )

### building vocab with tokenized words
w2v_model2.build_vocab(sent, progress_per=10000) 


###training the model on the dataset
w2v_model2.train(sent, total_examples=w2v_model2.corpus_count, epochs=30, report_delay=1)


### most similar words example
w2v_model2.wv.most_similar(["python"])


[('python_r', 0.9402478337287903),
 ('scala', 0.9381452202796936),
 ('programming_languages', 0.9356119632720947),
 ('java', 0.9177874326705933),
 ('kotlin', 0.8978232145309448),
 ('javascript', 0.8949280381202698),
 ('proficient', 0.8912734389305115),
 ('r', 0.871959388256073),
 ('programming', 0.8718017339706421),
 ('pyspark', 0.8647997379302979)]

## Vectorizer

In [19]:
def vectorizer(text):
    '''
    Replace the text with the respective vectors if there are in the model vocabulary
    '''
    new_text = []
    for word in text:
        if word in w2v_model.wv.vocab:
            vector = w2v_model.wv.__getitem__(word)
            new_text.append(vector)
    
    return new_text
    

In [22]:
df["vectorized_jobs"] = df["job_text_tokenized"].apply(vectorizer)

## Quick test for Translation pipeline and saving code for posterity


In [14]:
### quick test for transformer

from transformers import pipeline

summarizer = pipeline("summarization")

summarizer(df["job_text"][10],min_length=120, max_length=120)

All model checkpoint layers were used when initializing TFT5Model.

All the layers of TFT5Model were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5Model for predictions without further training.
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
Token indices sequence length is longer than the specified maximum sequence length for this model (724 > 512). Running this sequence through the model will result in indexing errors


[{'summary_text': 'MBition, Mercedes-Benz is bringing the automotive industry into the future . we are working on a software-defined architecture that allows us to create state-of-the-art customer functions . one of these high-performance domain controllers is used for automated driving functions (e.g. automated driving on highways) the company has been perfecting the art of moving people, products and services from A to B for more than 120 years . with an international team of 500 employees, we want to integrate people who appreciate quality,»»» .'}]

In [ ]:
### vector extraction from transfomer model



feature_extractor = pipeline("feature-extraction", model = "distilbert-base-cased")

def similarity(s1, s2):
    return  1 - spatial.distance.cosine(feature_extractor(s1)[0][-1], feature_extractor(s2)[0][-1])

def get_features(s):
    return feature_extractor(s)[0][-1]

sentance1 = "no one loves sushi"
sentance2 = "I use java for backend stuff and I'm important"
sentance3 = "I use html and css for be the frontend guy there is"

print(similarity(sentance1, sentance2))
print(similarity(sentance2, sentance3))


# modeling
from sklearn.cluster import KMeans
model  = KMeans(n_clusters=2)
X= np.array([get_features(sentance1),get_features(sentance2),get_features(sentance3)])
model.fit(X)
model.predict(X)
tokens = s.lower().replace('  ',' ').replace('\n',' ').split(' ')
threshold = 0.8
for token in tqdm(set(tokens)):
    if threshold < similarity(token.lower(), 'Skills'.lower()):
        print(token)


## only recommending the skills that are in the dict

In [195]:
def most_similar_skills(no_recom,dictionary,query):
    '''
    This function show the most similar skills to the input,which are also within the tech skills dictionary
    '''
    ### Turn the dictionary into a comprehensive list of all terms
    term_list = []
    for cat in dictionary.keys():
        for word in dictionary[cat]:
            term_list.append(word)

   ### Fetch similar words from the w2v model
    try:
        model_skills = w2v_model.wv.most_similar(query,topn= 100)
        
    
   ### only get the words without the distance measure 
        skill_words = []
        for i in range(len(model_skills)):
            skill_words.append(model_skills[i][0])
    
        return [skill for skill in skill_words if skill in term_list][0:no_recom] 
    except:
        print("Sorry,word not found")
no_recom = 3

with open("../fydjob/data/dicts/skills_dict.json") as json_file:
    dictionary= json.load(json_file)

query = ["leadership"]


most_similar_skills(no_recom,dictionary,query)

['organization', 'strategic', 'collaboration']

In [198]:
no_recom = 3

with open("../fydjob/data/dicts/skills_dict.json") as json_file:
    dictionary= json.load(json_file)

query = ["data science"]

most_similar_skills(no_recom,dictionary,query)

Sorry,word not found


## Tagging the skills/term with the category

In [157]:
business_dict = skills["business"]
knowledge_dict = skills["knowledge"]
programming_dict = skills["programming"]
soft_skills_dict = skills["soft_skills"]
tech_adjectives_dict = skills["tech_adjectives"]

    
def category_tagger(x):
    '''
    This function assigns the respective category to the recommended list
    '''
    if x in business_dict:
        return (x,"business")
    if x in knowledge_dict:
        return (x,"knowledge")
    if x in programming_dict:
        return (x,"programming")
    if x in soft_skills_dict:
        return (x,"soft_skills")
    if x in tech_adjectives_dict:
        return (x,"tech_adjectives")


[category_tagger(i) for i in test_query]

[('theano', 'programming'),
 ('sklearn', 'programming'),
 ('pytorch', 'programming'),
 ('scikitlearn', 'programming'),
 ('cntk', 'programming'),
 ('tensorflow', 'programming'),
 ('spacy', 'programming'),
 ('caffe', 'programming'),
 ('pandas', 'programming'),
 ('mathematics', 'knowledge'),
 ('statistics', 'knowledge'),
 ('nltk', 'programming')]

In [168]:
from nltk.tag import pos_tag

def noun_extractor(text,pos):
    '''
    This function takes a text and returns only the nouns in the text.
    It can be adjusted to any POS by changing the tag below.
    '''
    new_text = []
    for sentence in text:
        tagged_sentence = pos_tag(sentence)
        new_text.append([word for word,tag in tagged_sentence if tag in (pos)]) #change me!
    return new_text
    
pos = "NN"
df["job_text_sent"] =df["job_text_sent"].apply(noun_extractor)
              

            
       
        
    


KeyError: 'job_text_sent'

## Word2vec trained exclusivley on nouns

In [26]:
### extract nouns

df["job_text_sent"] = df["job_text_sent"].apply(noun_extractor)

In [15]:
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=20,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     )

### building vocab with tokenized words
w2v_model.build_vocab(df["job_text_sent"], progress_per=10000) 


###training the model on the dataset
w2v_model.train(df["job_text_sent"], total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)


### most similar words example
w2v_model.wv.most_similar(["python"])


TypeError: unhashable type: 'list'

In [7]:
pip install -e fydjob

ERROR: fydjob is not a valid editable requirement. It should either be a path to a local project or a VCS URL (beginning with bzr+http, bzr+https, bzr+ssh, bzr+sftp, bzr+ftp, bzr+lp, bzr+file, git+http, git+https, git+ssh, git+git, git+file, hg+file, hg+http, hg+https, hg+ssh, hg+static-http, svn+ssh, svn+http, svn+https, svn+svn, svn+file).
Note: you may need to restart the kernel to use updated packages.


In [6]:
from fydjob.Word2VecPipline import WordPipeline

ModuleNotFoundError: No module named 'fydjob'